# Simple Workflow with pyiron_base

## Define workflow with pyiron_base

In [1]:
from pyiron_base import Project, job

In [2]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [3]:
from simple_workflow import (
    add_x_and_y as _add_x_and_y, 
    add_x_and_y_and_z as _add_x_and_y_and_z,
)

In [4]:
workflow_json_filename = "pyiron_base_simple.json"

In [5]:
add_x_and_y = job(_add_x_and_y, output_key_lst=["x", "y", "z"])
add_x_and_y_and_z = job(_add_x_and_y_and_z)

In [6]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
obj = add_x_and_y(x=1, y=2, pyiron_project=pr)

In [8]:
w = add_x_and_y_and_z(x=obj.output.x, y=obj.output.y, z=obj.output.z, pyiron_project=pr)

In [9]:
write_workflow_json(delayed_object=w, file_name=workflow_json_filename)

In [10]:
!cat {workflow_json_filename}

{"nodes": {"0": "simple_workflow.add_x_and_y_and_z", "1": "simple_workflow.add_x_and_y", "2": 1, "3": 2}, "edges": [{"tn": 0, "th": "x", "sn": 1, "sh": "x"}, {"tn": 1, "th": "x", "sn": 2, "sh": null}, {"tn": 1, "th": "y", "sn": 3, "sh": null}, {"tn": 0, "th": "y", "sn": 1, "sh": "y"}, {"tn": 0, "th": "z", "sn": 1, "sh": "z"}]}

## Load Workflow with aiida

In [11]:
from aiida import load_profile

load_profile()

Profile<uuid='8bb81b57acda4bf6bbc69d5eb5ae123e' name='pwd'>

In [12]:
from python_workflow_definition.aiida import load_workflow_json

In [13]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [14]:

wg.run()

04/02/2025 02:36:58 PM <248> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3|WorkGraphEngine|continue_workgraph]: tasks ready to run: add_x_and_y2
04/02/2025 02:36:59 PM <248> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3|WorkGraphEngine|update_task_state]: Task: add_x_and_y2, type: PyFunction, finished.
04/02/2025 02:36:59 PM <248> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3|WorkGraphEngine|continue_workgraph]: tasks ready to run: add_x_and_y_and_z1
04/02/2025 02:37:00 PM <248> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3|WorkGraphEngine|update_task_state]: Task: add_x_and_y_and_z1, type: PyFunction, finished.
04/02/2025 02:37:00 PM <248> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
04/02/2025 02:37:00 PM <248> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3|WorkGraphEngine|finalize]: F

## Load Workflow with jobflow

In [15]:
from python_workflow_definition.jobflow import load_workflow_json

/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [16]:
from jobflow.managers.local import run_locally

In [17]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [18]:
result = run_locally(flow)
result

2025-04-02 14:37:00,813 INFO Started executing jobs locally
2025-04-02 14:37:01,234 INFO Starting job - add_x_and_y (3cb63b3e-289b-4c08-a9e4-0ba051181adb)
2025-04-02 14:37:01,238 INFO Finished job - add_x_and_y (3cb63b3e-289b-4c08-a9e4-0ba051181adb)
2025-04-02 14:37:01,239 INFO Starting job - add_x_and_y_and_z (bff452ed-d3b3-4d6c-8162-feaa188b5660)
2025-04-02 14:37:01,242 INFO Finished job - add_x_and_y_and_z (bff452ed-d3b3-4d6c-8162-feaa188b5660)
2025-04-02 14:37:01,243 INFO Finished executing jobs locally


{'3cb63b3e-289b-4c08-a9e4-0ba051181adb': {1: Response(output={'x': 1, 'y': 2, 'z': 3}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 'bff452ed-d3b3-4d6c-8162-feaa188b5660': {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))}}